# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

629

In [3]:
# Create empty data frame and add columns for city, country, latitude and longitude
cities_list = pd.DataFrame()
cities_list["City"] = ""
cities_list["Country"] = ""
# Randomly generate numbers for latitude and longitude and loop through 500 times
cities_list["Latitude"] = [np.random.uniform(-90, 90) for x in range(1500)]
cities_list["Longitude"] = [np.random.uniform(-180, 180) for x in range(1500)]
# Use latitude and longitude to find city and country by using interrows and citipy
for index, row in cities_list.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    cities_list.set_value(index, "City", citipy.nearest_city(lat, lng).city_name)
    cities_list.set_value(index, "Country", citipy.nearest_city(lat, lng).country_code)
# Delete any possible duplicate cities and reset index
cities_list = cities_list.drop_duplicates(["City", "Country"])
cities_list = cities_list.reset_index()
# Update data frame to just city and country column
cities_list = cities_list[["City", "Country"]]
cities_list = cities_list.iloc[:500]
cities_list

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]


,City,Country
0,kaitangata,nz
1,busselton,au
2,bilibino,ru
3,albany,au
4,atuona,pf
5,mahebourg,mu
6,khatanga,ru
7,timbiqui,co
8,amderma,ru
9,agadez,ne


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
# Perform API calls and display results in Data Frame
cities_list["Cloudiness"] = ""
cities_list["Date"] = ""
cities_list["Humidity"] = ""
cities_list["Latitude"] = ""
cities_list["Longitude"] = ""
cities_list["Max Temperature"] = ""
cities_list["Wind Speed"] = ""

# Create row counter and set it to one
row_counter = 1
# Create a dictionary with parameters
settings = {"appid": api_keys, "units": "imperial"}
# Create a base url
base_url = "http://api.openweathermap.org/data/2.5/weather?"
# Create query url
query_url = f"{base_url}appid={settings['appid']}&units={settings['units']}&q="

print("Beginning Data Retrieval")
print("---------------------------")
# Iterrows in to loop city row in query url
for index, row in cities_list.iterrows():
    try:
        cities_response = requests.get(query_url + row["City"] + "," + row["Country"])
        print(f"Processing Record {row_counter} | {row['City']}, {row['Country']}")
        print(cities_response.url)
        # Update row counter
        row_counter += 1
        # Turn cities response into json with owm module
        cities_response = owm.get_current(row["City"], **settings)
    
        # Create summary variables to retrieve certain data from cities response
        lat_summary = ["coord.lat"]
        lon_summary = ["coord.lon"]
        temp_summary = ["main.temp_max"]
        humidity_summary = ["main.humidity"]
        cloud_summary = ["clouds.all"]
        wind_summary = ["wind.speed"]
        date_summary = ["dt"]
    
        # Add retrieved data into cities_list data frame
        cities_list.set_value(index, "Cloudiness", cities_response(*cloud_summary))
        cities_list.set_value(index, "Date", cities_response(*date_summary))
        cities_list.set_value(index, "Humidity", cities_response(*humidity_summary))
        cities_list.set_value(index, "Latitude", cities_response(*lat_summary))
        cities_list.set_value(index, "Longitude", cities_response(*lon_summary))
        cities_list.set_value(index, "Max Temperature", cities_response(*temp_summary))
        cities_list.set_value(index, "Wind Speed", cities_response(*wind_summary))
    except:
        print(f"City not found. Skipping... {row['City']}, {row['Country']}")
        continue
    
print("---------------------------")
print("Data Retrieval Complete")
print("---------------------------")

Beginning Data Retrieval
---------------------------
Processing Record 1 | kaitangata, nz
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=kaitangata,nz
City not found. Skipping... kaitangata, nz
Processing Record 2 | busselton, au
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=busselton,au
City not found. Skipping... busselton, au
Processing Record 3 | bilibino, ru
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=bilibino,ru
City not found. Skipping... bilibino, ru
Processing Record 4 | a

Processing Record 28 | nizhneyansk, ru
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=nizhneyansk,ru
City not found. Skipping... nizhneyansk, ru
Processing Record 29 | pathein, mm
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=pathein,mm
City not found. Skipping... pathein, mm
Processing Record 30 | zelenoborsk, ru
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=zelenoborsk,ru
City not found. Skipping... zelenoborsk, ru
Processing Record 31 | port lincoln, au
http://api.openweathermap.o

Processing Record 55 | georgetown, sh
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=georgetown,sh
City not found. Skipping... georgetown, sh
Processing Record 56 | tiksi, ru
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=tiksi,ru
City not found. Skipping... tiksi, ru
Processing Record 57 | xai-xai, mz
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=xai-xai,mz
City not found. Skipping... xai-xai, mz
Processing Record 58 | tuktoyaktuk, ca
http://api.openweathermap.org/data/2.5/weather?ap

Processing Record 84 | potiskum, ng
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=potiskum,ng
City not found. Skipping... potiskum, ng
Processing Record 85 | grindavik, is
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=grindavik,is
City not found. Skipping... grindavik, is
Processing Record 86 | cidreira, br
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=cidreira,br
City not found. Skipping... cidreira, br
Processing Record 87 | antalaha, mg
http://api.openweathermap.org/data/2.5/weat

Processing Record 113 | okhotsk, ru
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=okhotsk,ru
City not found. Skipping... okhotsk, ru
Processing Record 114 | taltal, cl
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=taltal,cl
City not found. Skipping... taltal, cl
Processing Record 115 | rawson, ar
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=rawson,ar
City not found. Skipping... rawson, ar
Processing Record 116 | lanzhou, cn
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmo

...

Processing Record 364 | dakar, sn
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=dakar,sn
City not found. Skipping... dakar, sn
Processing Record 365 | sitka, us
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=sitka,us
City not found. Skipping... sitka, us
Processing Record 366 | chilca, pe
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=chilca,pe
City not found. Skipping... chilca, pe
Processing Record 367 | saint-paul, re
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%2

Processing Record 391 | kathmandu, np
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=kathmandu,np
City not found. Skipping... kathmandu, np
Processing Record 392 | aasiaat, gl
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=aasiaat,gl
City not found. Skipping... aasiaat, gl
Processing Record 393 | bedford, ca
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=bedford,ca
City not found. Skipping... bedford, ca
Processing Record 394 | chagda, ru
http://api.openweathermap.org/data/2.5/weather?

Processing Record 419 | brokopondo, sr
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=brokopondo,sr
City not found. Skipping... brokopondo, sr
Processing Record 420 | grand gaube, mu
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=grand%20gaube,mu
City not found. Skipping... grand gaube, mu
Processing Record 421 | la reforma, mx
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=la%20reforma,mx
City not found. Skipping... la reforma, mx
Processing Record 422 | hervey bay, au
http://api.open

Processing Record 448 | riyadh, sa
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=riyadh,sa
City not found. Skipping... riyadh, sa
Processing Record 449 | male, mv
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=male,mv
City not found. Skipping... male, mv
Processing Record 450 | murgab, tm
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=murgab,tm
City not found. Skipping... murgab, tm
Processing Record 451 | alofi, nu
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api

Processing Record 475 | coquimbo, cl
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=coquimbo,cl
City not found. Skipping... coquimbo, cl
Processing Record 476 | morro bay, us
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=morro%20bay,us
City not found. Skipping... morro bay, us
Processing Record 477 | tabukiniberu, ki
http://api.openweathermap.org/data/2.5/weather?appid=%3Cmodule%20'api_keys'%20from%20'/Users/katherinemorriss/Desktop/UKED201808DATA5/06-Python-APIs/Homework/Instructions/starter_code/api_keys.py'%3E&units=imperial&q=tabukiniberu,ki
City not found. Skipping... tabukiniberu, ki
Processing Record 478 | liwale, tz
http://api.openweathermap.o

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
cities_list


,City,Country,Cloudiness,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,kaitangata,nz,,,,,,,
1,busselton,au,,,,,,,
2,bilibino,ru,,,,,,,
3,albany,au,,,,,,,
4,atuona,pf,,,,,,,
5,mahebourg,mu,,,,,,,
6,khatanga,ru,,,,,,,
7,timbiqui,co,,,,,,,
8,amderma,ru,,,,,,,
9,agadez,ne,,,,,,,


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [6]:
# Temperature (F) vs. Latitude
timestamp = cities_list["Date"][0]
converted_timestamp = datetime.utcfromtimestamp(timestamp)
shortened_timestamp = datetime.date(converted_timestamp)

Lat = pd.to_numeric(cities_list["Latitude"])
Temp = pd.to_numeric(cities_list["Max Temperature"])

with sns.axes_style("darkgrid"):
    sns.regplot(data=cities_list, x=Lat, y=Temp, fit_reg=False)
plt.title(f"Latitude vs. Temperature ({shortened_timestamp})")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.scatter(x=Lat, y=Temp, color="blue", alpha=1, edgecolor="black")
plt.show()

NameError: name 'datetime' is not defined

#### Latitude vs. Humidity Plot

In [ ]:
plt.scatter(selected_cities["Latitude"],selected_cities["Humidity"], alpha = 0.75)

plt.title("Latitude vs. Humidity")
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.savefig("Humidity.png")
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(selected_cities["Latitude"],selected_cities["Cloudiness"], alpha = 0.75)

plt.title("Latitude vs. Cloudiness")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.savefig("Cloudiness.png")
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(selected_cities["Latitude"],selected_cities["Wind Speed"], alpha= 0.75)

plt.title("Latitude vs. Wind Speed")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.savefig("Wind Speed.png")
plt.show()